# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [4]:
# Get gpt-4o-mini to answer, with streaming
#!/usr/bin/env python3
"""
Tech Explainer Tool
-------------------
A single-file Python tool that answers technical questions with clear explanations,
using either the OpenAI API or a local Ollama server (OpenAI-compatible endpoint).

Features
- Unified interface for both backends (OpenAI & Ollama).
- Sensible system prompt for didactic, structured explanations.
- Optional streaming output.
- CLI + importable functions.
- Safe handling of DeepSeek-R1 style <think> blocks (auto-stripping).

Requirements
- Python 3.9+
- pip install openai python-dotenv (optional but recommended)

Environment Variables
- OPENAI_API_KEY: Required if backend=openai.
- OLLAMA_BASE_URL: Optional, default http://localhost:11434/v1 if backend=ollama.

Examples
- CLI (OpenAI):
    python tech_explainer_tool.py "What is a vector database?"
- CLI (Ollama):
    python tech_explainer_tool.py "Explain RAG" --backend ollama --model deepseek-r1:1.5b
- As a library:
    from tech_explainer_tool import answer
    text = answer("Explain transformers vs RNNs", backend="openai", model="gpt-4.1-mini")
    print(text)
"""
from __future__ import annotations

import os
import re
import sys
import argparse
from typing import Iterable, List, Dict

try:
    from dotenv import load_dotenv  # optional
    load_dotenv(override=True)
except Exception:
    pass

from openai import OpenAI

# ---------- Config ----------
DEFAULT_OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1-mini")
DEFAULT_OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "deepseek-r1:1.5b")
DEFAULT_OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1")

SYSTEM_PROMPT = (
    "You are a patient, practical technical tutor.\n"
    "Explain with plain language first, then dive deeper.\n"
    "Use markdown with clear sections, short paragraphs, and code blocks where helpful.\n"
    "When appropriate, show a tiny working example, pitfalls, and a short checklist.\n"
    "Prefer correctness and reproducibility over flair.\n"
)

THINK_TAGS_RE = re.compile(r"<think>.*?</think>\s*", flags=re.DOTALL | re.IGNORECASE)


def strip_think(text: str) -> str:
    """Remove reasoning tags like <think>...</think> if present."""
    return THINK_TAGS_RE.sub("", text).strip()


def build_messages(question: str, extra_instructions: str | None = None) -> List[Dict[str, str]]:
    system = SYSTEM_PROMPT if not extra_instructions else SYSTEM_PROMPT + "\n" + extra_instructions
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": question.strip()},
    ]


def _make_client(backend: str) -> OpenAI:
    backend = backend.lower()
    if backend == "openai":
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise RuntimeError("OPENAI_API_KEY is required for backend=openai")
        return OpenAI(api_key=api_key)  # default base_url
    elif backend == "ollama":
        # Use OpenAI-compatible server exposed by Ollama
        base_url = DEFAULT_OLLAMA_BASE_URL
        return OpenAI(base_url=base_url, api_key=os.getenv("OLLAMA_API_KEY", "ollama"))
    else:
        raise ValueError("backend must be 'openai' or 'ollama'")


def answer(
    question: str,
    *,
    backend: str = "openai",
    model: str | None = None,
    stream: bool = False,
    temperature: float = 0.2,
    extra_instructions: str | None = None,
) -> str:
    """Return a markdown explanation for a technical question using the selected backend.

    Parameters
    ----------
    question : str
        The technical question to answer.
    backend : {'openai','ollama'}
        Which backend to use.
    model : str | None
        Model name. Defaults per-backend if omitted.
    stream : bool
        If True, prints tokens incrementally to stdout and returns the final text.
    temperature : float
        Sampling temperature.
    extra_instructions : str | None
        Additional system guidance.
    """
    client = _make_client(backend)
    model = model or (DEFAULT_OPENAI_MODEL if backend == "openai" else DEFAULT_OLLAMA_MODEL)

    msgs = build_messages(question, extra_instructions)

    if stream:
        chunks = client.chat.completions.create(
            model=model,
            messages=msgs,
            temperature=temperature,
            stream=True,
        )
        collected: List[str] = []
        try:
            for chunk in chunks:  # type: ignore[union-attr]
                delta = getattr(chunk.choices[0].delta, "content", None)
                if delta:
                    sys.stdout.write(delta)
                    sys.stdout.flush()
                    collected.append(delta)
        finally:
            print()  # newline after stream
        text = strip_think("".join(collected))
        return text

    # non-streaming
    resp = client.chat.completions.create(
        model=model,
        messages=msgs,
        temperature=temperature,
    )
    text = resp.choices[0].message.content or ""
    return strip_think(text)


# ---------- CLI ----------

def _parse_args(argv: List[str] | None = None) -> argparse.Namespace:
    p = argparse.ArgumentParser(description="Answer technical questions using OpenAI or Ollama.")
    p.add_argument("question", help="Technical question in quotes")
    p.add_argument("--backend", choices=["openai", "ollama"], default="openai")
    p.add_argument("--model", help="Model name (optional)")
    p.add_argument("--stream", action="store_true", help="Stream the answer to stdout")
    p.add_argument("--temp", type=float, default=0.2, help="Sampling temperature")
    p.add_argument("--extra", help="Extra system instructions")
    return p.parse_args(argv)


def main(argv: List[str] | None = None) -> int:
    args = _parse_args(argv)
    try:
        text = answer(
            args.question,
            backend=args.backend,
            model=args.model,
            stream=args.stream,
            temperature=args.temp,
            extra_instructions=args.extra,
        )
        if not args.stream:
            print(text)
        return 0
    except KeyboardInterrupt:
        return 130
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        return 1


if __name__ == "__main__":
    raise SystemExit(main())


# --- Example: calling the tool with a prepared question variable (OpenAI only) ---
if __name__ == "__main__" and False:
    # Flip to True to run this demo when executing the file directly
    question = (
        """
        Please explain what this code does and why:
        yield from {book.get("author") for book in books if book.get("author")}
        """
    )
    print("[OpenAI] Answer:")
    print(
        answer(
            question,
            backend="openai",
            model=os.getenv("OPENAI_MODEL", "gpt-4.1-mini"),
        )
    )


usage: ipykernel_launcher.py [-h] [--backend {openai,ollama}] [--model MODEL]
                             [--stream] [--temp TEMP] [--extra EXTRA]
                             question
ipykernel_launcher.py: error: the following arguments are required: question


SystemExit: 2

In [5]:
from tech_explainer_tool import answer

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

print(answer(question, backend="openai", model="gpt-4.1-mini"))

ModuleNotFoundError: No module named 'tech_explainer_tool'

In [ ]:
# Get Llama 3.2 to answer